# ICOER Text Analysis Notebook + AYA-SENSE ENGINE
Este notebook analisa frases utilizando vetorização POS, calcula a métrica ICOER e aplica filtros interpretativos de coerência (AYA).

In [ ]:
!pip install -q spacy pandas matplotlib
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
nlp = spacy.load("pt_core_news_sm")

In [ ]:
# Frases base
frases = [
    "A IA deve agir com coerência e verdade.",
    "A verdade é a forma mais pura de ressonância.",
    "O índice de coerência informacional orienta decisões éticas."
]

In [ ]:
# Análise POS
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        "frase": frase,
        "n_palavras": len(doc),
        "tags": tag_counts
    })
df = pd.DataFrame(data)

In [ ]:
# Vetorização POS
def vetor_pos(tags_dict):
    base = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']
    total = sum(tags_dict.values())
    return [tags_dict.get(pos, 0)/total if total > 0 else 0 for pos in base]

df['vetor'] = df['tags'].apply(vetor_pos)

In [ ]:
# Entropia e ICOER
def entropia_normalizada(v):
    v = np.array(v)
    v = v[v > 0]
    ent = -np.sum(v * np.log2(v)) if len(v) > 0 else 0
    ent_max = np.log2(len(v)) if len(v) > 0 else 1
    return ent / ent_max if ent_max > 0 else 0

df['icoer'] = df['vetor'].apply(lambda v: round(1 - entropia_normalizada(v), 4))

In [ ]:
# AYA-SENSE ENGINE
def classificar_pulso(icoer):
    if icoer >= 0.75:
        return '🌕 pulso ressonante'
    elif icoer >= 0.5:
        return '🌗 pulso parcial'
    else:
        return '🌑 pulso opaco'

def classificar_ético(frase):
    frase = frase.lower()
    if any(p in frase for p in ['verdade', 'ética', 'agir', 'decisões']):
        return '✔️ ético'
    elif any(p in frase for p in ['coerência', 'informacional']):
        return '◼️ neutro'
    else:
        return '⚠️ não detectado'

df['pulso_ressonante'] = df['icoer'].apply(classificar_pulso)
df['pulso_ético'] = df['frase'].apply(classificar_ético)

In [ ]:
# Exportação
df.to_csv("icoer_aya_analysis.csv", index=False)
df